In [1]:
import os
import json
import shutil
from qa_utils import load_jsonl
import pandas as pd

In [2]:

val_trusted_list = ["True", "False"]
val_target_list = ["Yes","No"]
val_question_list = ['obviously ']
SEQUENCES = "tft"
TEMPLATES = "normal"


In [5]:
dst_folder = r"prompts/"
dst_file =  dst_folder +  "LAMA_relations_" + name_pre + ".jsonl"

In [28]:

def get_dic_questions(case):
    
    lama_relations_path = 'prompts/LAMA_relations_'+str(case)+'.jsonl'
    lama_relations = load_jsonl(lama_relations_path)
    questions = []
    for i in lama_relations:
        #Possible to add the fact
        #print(i['template'])
        questions.append(i['template'])

    dict_questions = {key: [] for key in questions}
    #print("*"*10)
    #print("dict_questions")
    #print(dict_questions)
    #print("*"*10)
    return dict_questions

def extract_acc(dictionary_values, case):

    # getting dict with questions as keys, and empty of values
    dict_questions = get_dic_questions(case)
    for d in dictionary_values:
        #print(d)
        #print("*"*10)
        #print("D VALUES")
        #print(d)
        #print("*"*10)
        for i in dictionary_values[d]:
            data  = [i[0][2], i[0][3], i[0][4],d]
            dict_questions[i[0][0]].append(data)

    final_list = []
    #print("="*10)
    #print(len(dict_questions))
    #print(dict_questions)
    c = 0
    for dict_q in dict_questions:
        #print(dict_questions[dict_q][0][3])
        #print("ACAAAAAAAAAAAAAAAAA")
        #print("/"*10)
        #print(dict_q)
        #print(dict_questions[dict_q])
        #print("/"*10)
        #print(dict_questions[dict_q][0])
        #print("/"*10)
        #print(dict_questions[dict_q][0][0])
        #print("/"*10)
        #print(dict_questions[dict_q][0][1]*2)
        acc = dict_questions[dict_q][0][0]/(dict_questions[dict_q][0][1]*2)
        #print("/acc"*10)
        #print(acc)
        final_list.append([dict_q, acc, dict_questions[dict_q][0][2],  dict_questions[dict_q][0][3]])
        c +=1
    return final_list


In [31]:
for i in range(len(val_trusted_list)):
    for val_question in val_question_list:
        # file names
        fname_df = str(val_trusted_list[i]) + "_" + str(val_target_list[i])
        #src_file = src_folder + "LAMA_relations.jsonl"
        val_q = val_question.replace(" ", "_")
        name_pre = str(val_trusted_list[i] + "_" + val_target_list[i] + "_" + val_q)
        dst_file = dst_folder + "LAMA_relations_" + name_pre
        
        #shutil.copyfile(src_file, dst_file)

        # reading the data from the file
        with open('results/'+fname_df+".txt") as f:
            data = f.read()
            
        #print("DATA")
        #print(data)
            
        # reconstructing the data as a dictionary
        final_quest = json.loads(data)
        #print(final_quest)
        final_list = extract_acc(final_quest, name_pre)
        print("FINAL LIST", final_list)
        df_final_list = pd.DataFrame(final_list, columns = ['question', 'acc', 'target','fact'])
        print("DF FINAL LIST",df_final_list)
        fname_df_final = fname_df + val_question + "_dataframe.csv"
        df_final_list.to_csv(fname_df_final)
        df_final_list.to_csv("results/"+fname_df_final)


FINAL LIST [['Does [P103_TA] speak [P103_TB] as a mother tongue? Yes. Does [P103_FA] speak [P103_FB] as a mother tongue? No. Does [P103_TA] speak [P103_TB] as a mother tongue? Yes. Does [W] speak [X] as a mother tongue? obviously [Y]', 0.49129989764585463, 'Yes', 'P103'], ['Does [P103_TA] speak [P103_TB] as a native language? Yes. Does [P103_FA] speak [P103_FB] as a native language? No. Does [P103_TA] speak [P103_TB] as a native language? Yes. Does [W] speak [X] as a native language? obviously [Y]', 0.48771750255885365, 'Yes', 'P103'], ["Is [P103_TB] [P103_TA]'s mother tongue? Yes. Is [P103_FB] [P103_FA]'s mother tongue? No. Is [P103_TB] [P103_TA]'s mother tongue? Yes. Is [X] [W]'s mother tongue? obviously [Y]", 0.4938587512794268, 'Yes', 'P103'], ["Is [P103_TB] [P103_TA]'s native language? Yes. Is [P103_FB] [P103_FA]'s native language? No. Is [P103_TB] [P103_TA]'s native language? Yes. Is [X] [W]'s native language? obviously [Y]", 0.49641760491299897, 'Yes', 'P103'], ['Does [P108_TA] 

In [32]:

# lo de arriba funciona bien
# Merging and obtaining final accuracies
for val_question in val_question_list:

    # Obtaining: True_yes_dataframe.csv
    positive_side  = val_trusted_list[0] + "_" + val_target_list[0] + val_question + "_dataframe.csv"
    positive_side_df = pd.read_csv("results/"+positive_side)

    # Obtaining: False_no_dataframe.csv
    negative_side  = val_trusted_list[1] + "_" + val_target_list[1] + val_question + "_dataframe.csv"
    negative_side_df = pd.read_csv("results/"+negative_side)

    df_master = pd.merge(positive_side_df, negative_side_df, on='question')
    #df_master.head()
    df_master['accuracy'] = df_master['acc_x'] + df_master['acc_y']
    #df_master.head()
    df_master = df_master.sort_values(by=['fact_x'])
    #df_master.head(20)

    # is the same to use fact_x and fact_y
    df_group = df_master.groupby(['fact_x']).max()
    df_group.head()

    df_final = df_group[['question', 'accuracy']]
    print(df_final.head())

    df_final.to_csv("results/"+"question_"+val_question+".csv")


                                                 question  accuracy
fact_x                                                             
P103    Is [P103_TB] [P103_TA]'s native language? Yes....  0.496929
P108    Is [P108_TA] employed by [P108_TB]? Yes. Is [P...  0.528721
P19     Was [P19_TB] [P19_TA]'s place of birth? Yes. W...  0.498941
P20     Was [P20_TB] [P20_TA]'s place of death? Yes. W...  0.498426
P279    Is [P279_TB] the class of [P279_TA]? Yes. Is [...  0.542012
